In [ ]:
# !wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt \
#      --output-document=../data/tinyshakespeare.txt

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
from torch import Tensor


In [ ]:
device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

In [ ]:
raw_text_path = "../data/tinyshakespeare.txt"

In [ ]:
with open(raw_text_path, "r", encoding="utf-8") as f:
    text = f.read()


In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(*chars)

In [ ]:
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

In [ ]:
print(encode("hi there"))
print(decode(encode("hi theere")))

In [ ]:
data = torch.tensor(encode(text), dtype=torch.int64)
print(data.shape, data.dtype)

In [ ]:
print(len(data))
n = int(0.9 * len(data))
train_data = data[0:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size + 1]

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size + 1]

for t in range(block_size):
    context = x[: t + 1]
    target = y[t]
    print(f"When input is {context}, the target is: {target}")

In [ ]:
torch.manual_seed(1337)
batch_size = 4  # number of sequences we will process in parallel
block_size = 8  # maximum context length for predictions


def get_batch(split: str) -> tuple[Tensor, Tensor]:
    # generate a small batch of data for inputs x and targets y
    data = train_data if split == "train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i : i + block_size] for i in ix])
    y = torch.stack([data[i + 1 : i + block_size + 1] for i in ix])
    return x, y


xb, yb = get_batch("train")

print(
    f"inputs:\n"
    f"{xb.shape}\n"
    f"{xb}\n"
    f"targets:\n"
    f"{yb.shape}\n"
    f"{yb}\n"
    f"-----------"
)

for b in range(batch_size):
    for t in range(block_size):
        context = xb[b, : t + 1]
        target = yb[b, t]
        print(f"when input is {context.tolist()}, the target is: {target}")

In [ ]:
print(xb) # input for the transformer

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None) -> tuple[Tensor, Tensor]:
        logits = self.token_embedding_table(idx)  # (B, T, C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B * T, C)
            targets = targets.view(B * T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is the (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :]  # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)  # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1)  # (B, 1)
            idx = torch.cat((idx, idx_next), dim=1)  # (B, T + 1)
        return idx


model = BigramLanguageModel(vocab_size) # .to(device)
print(next(model.parameters()).device)

logits, loss = model(xb, yb)
print(logits.shape)
print(loss)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
print(type(optimizer).mro())

# train_data = train_data.to(device)

In [ ]:
batch_size = 32
for steps in range(10000):
    xb, yb = get_batch("train")
    
    logits, loss = model(xb, yb)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

print(loss.item())

In [ ]:
print(
    decode(
        model.generate(idx=torch.zeros((1, 1), dtype=torch.int64), max_new_tokens=100)[
            0
        ].tolist()
    )
)